In [1]:
import pandas as pd
import geopandas as gpd
pd.set_option('display.max_columns', None)

### Spatial Join

In [2]:
# Read US Tract in 1940
tract_1940 = gpd.read_file(r'Data\US_tract_1940.shp')

# length of tract_1940
print(tract_1940.shape[0])

# CRS of tract_1940
print(tract_1940.crs) # esri 102003

# Calculate area for each tract
tract_1940['area_km2_1940_tract'] = tract_1940.geometry.area / 1000000

# First 2 rows
tract_1940.head(2)

7563
PROJCS["USA_Contiguous_Albers_Equal_Area_Conic",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",37.5],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["ESRI","102003"]]


,NHGISST,NHGISCTY,GISJOIN,GISJOIN2,SHAPE_AREA,SHAPE_LEN,geometry,area_km2_1940_tract
0,550,0790,G55007900090,55007900090,1.902826e+06,9640.965368,"POLYGON ((650791.784 654746.221, 650886.198 65...",1.902826
1,550,0790,G55007900094,55007900094,4.469542e+06,12723.051874,"POLYGON ((646759.565 651779.856, 646851.837 65...",4.469542


In [3]:
# Read 2020 tract
tract_2020 = gpd.read_file(r'Data\US_tract_2020.shp')
print(tract_2020.shape[0]) 

# CRS 
print(tract_2020.crs)      
tract_2020 = tract_2020.to_crs("ESRI:102003")
print(tract_2020.crs)

tract_2020.head(2)         

84414
EPSG:4269
ESRI:102003


,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,01,045,020801,01045020801,208.01,Census Tract 208.01,G5020,S,19261326.0,0.0,+31.3971437,-085.6615959,"POLYGON ((974088.803 -630642.090, 974105.477 -..."
1,01,017,954200,01017954200,9542,Census Tract 9542,G5020,S,62998696.0,34641.0,+32.8341436,-085.2678516,"POLYGON ((992602.687 -469304.620, 992744.903 -..."


In [4]:
# Spatial join
# If the centroid of an 2020 tract is in 1940 tract, the 2020 tract uses the same socioeconomic info of 1940 tract

# Create centroid of tract_2020
tract_2020['centroid'] = tract_2020.geometry.centroid
centroid_2020 = tract_2020.set_geometry('centroid')

# Spatial join - join info from tract_1940 to centroid of tract_2020
joined = gpd.sjoin(centroid_2020, tract_1940, op='within', how='left')

# Keep those joined
joined_cleaned = joined[joined['index_right'].notna()]
cols_keep = ['GEOID', 'geometry', 'GISJOIN', 'area_km2_1940_tract']
joined_cleaned = joined_cleaned[cols_keep]

# Summarize how many tracts are joined
print('# 2020 tracts', joined.shape[0])
print('# 2020 tracts of which the centroids are within 1940 tracts', joined_cleaned.shape[0]) 

joined_cleaned.head(2)

C:\Users\hw2298\AppData\Local\anaconda3\envs\hanxue_env\lib\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


# 2020 tracts 84414
# 2020 tracts of which the centroids are within 1940 tracts 12925


,GEOID,geometry,GISJOIN,area_km2_1940_tract
135,01073004502,"POLYGON ((845982.067 -405950.604, 845988.530 -...",G01007300044,1.282831
136,01073004501,"POLYGON ((846272.310 -407111.307, 846303.138 -...",G01007300049,1.715927


### Join 1940 socioeconomic data & HOLC data

### 1940 socioeconomic data

In [5]:
info = pd.read_csv(r'Data\nhgis_1940_tract.csv')
print(info.shape[0])
info.head(2)

# no black & other race info; use non white
info = info.iloc[1:]

for col in info.columns[10:]: 
    info[col] = pd.to_numeric(info[col], errors='coerce')
    
info['pct_black'] = 100 * info['BVG001'] / info['BUB001'] 
info['pct_nonwhite'] = 100 * info['BUQ002'] / info['BUB001']
info['pct_foreign_white'] = 100 * (info['BVT003'] + info['BVT004']) / info['BUQ001']
info['pct_college_above'] = 100 * (info['BUH007'] + info['BUH008'] + info['BUH016'] + info['BUH017']) / info['BUB001']

# unempl: 
# 1 - (Male In labor force: Employed  + Female In labor force: Employed )/ (male in labor force + female in labor force)
info['pct_unempl'] = 100 * (1 - ((info['BUX001'] + info['BUX009']) / (info['BUW001'] + info['BUW003'])))

info['pct_renter']= 100 * info['BU2002'] / (info['BU2001'] + info['BU2002'])

info = info.rename(columns = {'BUB001':'total_pop', 'BVC001':'median_h_value'})

info['median_h_value'] = info['median_h_value'] / 1000

info = info[['GISJOIN', 'total_pop', 'pct_black', 'pct_nonwhite', 'pct_foreign_white', 
             'median_h_value', 'pct_college_above', 
             'pct_unempl', 'pct_renter']]

info['GISJOIN'] = info['GISJOIN'].str.zfill(12)
info.head()

7468


,GISJOIN,total_pop,pct_black,pct_nonwhite,pct_foreign_white,median_h_value,pct_college_above,pct_unempl,pct_renter
1,G01007300001,2792,0.322350,0.322350,0.431189,2.435,5.336676,13.008937,50.573066
2,G01007300002,1796,1.614699,1.614699,1.358234,2.444,4.398664,16.268657,58.298755
3,G01007300003,3935,4.040661,4.040661,0.582627,2.685,6.429479,12.204234,62.118321
4,G01007300004,4882,15.116755,15.116755,0.579151,2.284,2.929127,14.592275,51.386623
5,G01007300005,5960,27.449664,27.449664,0.994450,1.842,2.953020,18.115002,60.804672


In [6]:
# Some further cleaning

# Median household value
print('0 home value--removed', info[info.median_h_value == 0].shape[0])
info = info[info.median_h_value != 0]
print(info.shape[0])

# Pct foreign white
print(info.isna().sum())
# remove nan values for pct_foreign_white--2
info = info[info.pct_foreign_white.notna()]
print(info.shape[0])

0 home value--removed 1408
6059
GISJOIN              0
total_pop            0
pct_black            0
pct_nonwhite         0
pct_foreign_white    2
median_h_value       0
pct_college_above    0
pct_unempl           0
pct_renter           0
dtype: int64
6057


In [7]:
# Join with tracts
# name of the datasets cleaned: joined_cleaned

joined_cleaned['GISJOIN'] = joined_cleaned['GISJOIN'].str.zfill(12)
df_join1 = pd.merge(joined_cleaned, info, on = 'GISJOIN', how = 'left')
print(df_join1.shape[0])
print('no data', df_join1[df_join1.total_pop.isnull()].shape[0])  # sociodemographic data does not cover all shapefile data in 1940.

df_join1['pop_dens'] = 0.001 * df_join1['total_pop'] / df_join1['area_km2_1940_tract'] # number of people in thousands / km2

df_join1 = df_join1[df_join1.total_pop.notnull()]
print('number of 2020 tracts after removing rows with no 1940 socioeconomic data', df_join1.shape[0])
df_join1.head(2)

12925
no data 1909
number of 2020 tracts after removing rows with no 1940 socioeconomic data 11016


,GEOID,geometry,GISJOIN,area_km2_1940_tract,total_pop,pct_black,pct_nonwhite,pct_foreign_white,median_h_value,pct_college_above,pct_unempl,pct_renter,pop_dens
1,01073004501,"POLYGON ((846272.310 -407111.307, 846303.138 -...",G01007300049,1.715927,9804.0,6.997144,6.997144,3.619215,3.821,15.942472,13.302398,84.439505,5.713530
6,01073003700,"POLYGON ((838057.862 -407828.934, 838223.241 -...",G01007300037,1.886692,4041.0,0.222717,0.222717,1.413690,3.691,13.288790,7.983917,46.752059,2.141843


### Get HOLC grade for each tract

###### spatial join HOLC and tract

In [9]:
# read tract and holc boundaries
tract_2020 = gpd.read_file(r'Data\US_tract_2020.shp')
tract_2020 = tract_2020.to_crs("ESRI:102003")
tract_2020['tract_area_m2'] = tract_2020['geometry'].area

holc = gpd.read_file(r'Data\holc_ad_data.shp')
print(holc.crs)
holc = holc.to_crs("ESRI:102003")

# Spatial join
tract_holc = gpd.overlay(tract_2020, holc, how='intersection')

# Filter 
# Threshold of assigning HOLC data to a tract
# Set it as _____ 25%

threshold = 0.25
tract_holc['intersect_area_m2'] = tract_holc['geometry'].area
tract_holc['area_pct'] = tract_holc['intersect_area_m2'] / tract_holc['tract_area_m2']

filtered_tract_holc = tract_holc.groupby(['GEOID'])['area_pct'].sum().reset_index()
lst_GEOID_keep = filtered_tract_holc[filtered_tract_holc.area_pct >= threshold].GEOID.to_list()

print('number of tracts that overlay with HOLC:', filtered_tract_holc.shape[0])
print('remove the tracts using threshold 25%:', len(lst_GEOID_keep)) 
print('percent of removal:', 100*(1-len(lst_GEOID_keep)/filtered_tract_holc.shape[0]), '%')

EPSG:4326
number of tracts that overlay with HOLC: 16599
remove the tracts using threshold 25%: 12853
percent of removal: 22.56762455569613 %


In [10]:
# determine which grade
tract_holc = tract_holc[tract_holc.GEOID.isin(lst_GEOID_keep)]
print(tract_holc.shape[0])  

# this is for situation such as one tract has 2 or more D grade pieces. 
# we need to sum them first to make sure one tract has at most 4-5 rows.
area_by_grade = tract_holc.groupby(['GEOID', 'holc_grade'])['area_pct'].sum().reset_index() # nan is excluded 

# Return index of first occurrence of maximum over requested axis.
idx = area_by_grade.groupby('GEOID')['area_pct'].idxmax()  
max_area_grade = area_by_grade.loc[idx][['GEOID', 'holc_grade']]
print(max_area_grade.shape[0])

# sanity check: 
# max_area_grade[max_area_grade.holc_grade.isnull()] # each GEOID is assigned with one grade

max_area_grade.head()

35033
12853


,GEOID,holc_grade
2,01073000100,D
4,01073000300,D
6,01073000400,D
7,01073000500,D
8,01073000700,D


In [11]:
max_area_grade.groupby('holc_grade').count()

,GEOID
holc_grade,
A,795
B,2568
C,5873
D,3609
E,8


In [12]:
total_n = max_area_grade.groupby('holc_grade').count().sum()
print(total_n)
100*max_area_grade.groupby('holc_grade').count()/total_n

GEOID    12853
dtype: int64


,GEOID
holc_grade,
A,6.185326
B,19.979771
C,45.693612
D,28.079048
E,0.062242


### Join 2020 tracts with 1940 socioeconomic data & 2020 tracts with 1940 HOLC grades

In [13]:
df_join2 = pd.merge(df_join1, max_area_grade, on = 'GEOID', how = 'left')
print('how many without holc grade info:', df_join2[df_join2.holc_grade.isnull()].shape[0])
df_join2 = df_join2[df_join2.holc_grade.notnull()]

# Remove grade E 
print('how many grade E:', df_join2[df_join2.holc_grade == 'E'].shape[0])
df_join2 = df_join2[df_join2.holc_grade != 'E']
print('after remove rows without holc grade info and rows with E:', df_join2.shape[0])
df_join2.head(2)

how many without holc grade info: 3045
how many grade E: 4
after remove rows without holc grade info and rows with E: 7967


,GEOID,geometry,GISJOIN,area_km2_1940_tract,total_pop,pct_black,pct_nonwhite,pct_foreign_white,median_h_value,pct_college_above,pct_unempl,pct_renter,pop_dens,holc_grade
0,01073004501,"POLYGON ((846272.310 -407111.307, 846303.138 -...",G01007300049,1.715927,9804.0,6.997144,6.997144,3.619215,3.821,15.942472,13.302398,84.439505,5.713530,D
1,01073003700,"POLYGON ((838057.862 -407828.934, 838223.241 -...",G01007300037,1.886692,4041.0,0.222717,0.222717,1.413690,3.691,13.288790,7.983917,46.752059,2.141843,C


### Join with health data

In [16]:
cdc = pd.read_csv('Data\PLACES_2022_release.csv')
print(cdc.shape[0])
cdc['TractFIPS'] = cdc['TractFIPS'].astype(str).str.zfill(11)
# cdc[cdc['OBESITY_CrudePrev'].isna()] # no nan values
cdc = cdc[['TractFIPS', 'StateAbbr', 'OBESITY_CrudePrev', 'DIABETES_CrudePrev', 'BPHIGH_CrudePrev']]
print(cdc[cdc['BPHIGH_CrudePrev'].isna()].shape[0]) # 1999 tracts lack hypertension info but not obesity and diabetes.
cdc.head(2)

72337
1999


,TractFIPS,StateAbbr,OBESITY_CrudePrev,DIABETES_CrudePrev,BPHIGH_CrudePrev
0,37051003402,NC,24.2,1.7,11.2
1,04013422643,AZ,28.3,6.4,23.7


In [17]:
# max_area_grade['GEOID'] = max_area_grade['GEOID'].astype(str).str.zfill(11)

df_join3 = pd.merge(df_join2, cdc, left_on = 'GEOID', right_on = 'TractFIPS', how = 'left')
print('tracts without health data:', df_join3[df_join3.TractFIPS.isna()].shape[0])

df_join3 = df_join3[df_join3.TractFIPS.notna()]
print('final data we use:', df_join3.shape[0])

df_join3.rename(columns = {'OBESITY_CrudePrev':'obesity', 
                           'DIABETES_CrudePrev':'diabetes', 
                           'BPHIGH_CrudePrev':'bphigh'}, inplace = True)
 
df_join3.head(2)

tracts without health data: 986
final data we use: 6981


,GEOID,geometry,GISJOIN,area_km2_1940_tract,total_pop,pct_black,pct_nonwhite,pct_foreign_white,median_h_value,pct_college_above,pct_unempl,pct_renter,pop_dens,holc_grade,TractFIPS,StateAbbr,obesity,diabetes,bphigh
1,01073003700,"POLYGON ((838057.862 -407828.934, 838223.241 -...",G01007300037,1.886692,4041.0,0.222717,0.222717,1.413690,3.691,13.288790,7.983917,46.752059,2.141843,C,01073003700,AL,53.2,21.1,51.4
2,01073000300,"POLYGON ((850720.399 -401753.886, 850764.512 -...",G01007300003,3.116637,3935.0,4.040661,4.040661,0.582627,2.685,6.429479,12.204234,62.118321,1.262579,D,01073000300,AL,52.1,21.8,49.3


### Add region

In [18]:
region_dict = {
    'CT': 'Northeast', 'ME': 'Northeast', 'MA': 'Northeast', 'NH': 'Northeast', 'NJ': 'Northeast', 'NY': 'Northeast', 'PA': 'Northeast', 'RI': 'Northeast', 'VT': 'Northeast',
    'IL': 'Midwest', 'IN': 'Midwest', 'IA': 'Midwest', 'KS': 'Midwest', 'MI': 'Midwest', 'MN': 'Midwest', 'MO': 'Midwest', 'NE': 'Midwest', 'ND': 'Midwest', 'OH': 'Midwest', 'SD': 'Midwest', 'WI': 'Midwest',
    'AL': 'South', 'AR': 'South', 'DE': 'South', 'FL': 'South', 'GA': 'South', 'KY': 'South', 'LA': 'South', 'MD': 'South', 'MS': 'South', 'NC': 'South', 'OK': 'South', 'SC': 'South', 'TN': 'South', 'TX': 'South', 'VA': 'South', 'WV': 'South',
    'AK': 'West', 'AZ': 'West', 'CA': 'West', 'CO': 'West', 'HI': 'West', 'ID': 'West', 'MT': 'West', 'NV': 'West', 'NM': 'West', 'OR': 'West', 'UT': 'West', 'WA': 'West', 'WY': 'West'
}

# Map the state abbreviations to the regions
df_join3['Region'] = df_join3['StateAbbr'].map(region_dict)

dummies = pd.get_dummies(df_join3['Region'], prefix='Region').astype(int) # add astype(int)
df_join3 = pd.concat([df_join3, dummies], axis=1)

df_join3.to_csv(r'Data\dataset.csv')

In [19]:
df_join3.groupby('holc_grade').describe()

area_km2_1940_tract                                           \
                         count      mean        std       min       25%   
holc_grade                                                                
A                        330.0  6.016211  12.000907  0.387524  1.457750   
B                       1337.0  3.258765   7.182198  0.202862  0.942136   
C                       3390.0  3.390278   5.697546  0.115095  0.793142   
D                       1924.0  2.536491   6.423494  0.086789  0.547859   

                                           total_pop               \
                 50%       75%         max     count         mean   
holc_grade                                                          
A           2.792338  6.029525  112.069304     330.0  5648.590909   
B           1.504696  2.874363  112.069304    1337.0  9358.000000   
C           1.607625  3.555747  112.069304    3390.0  9997.464602   
D           0.977015  2.094729  126.648909    1924.0  9613.190229   

                                                                   pct_black  \
                    std    min      25%     50%       75%      max     count   
holc_grade                                                                     
A           5092.004898  575.0  2815.50  4426.0   7064.75  34314.0     330.0   
B           9145.319054  457.0  3654.00  5443.0  10267.00  36178.0    1337.0   
C           8712.116977  537.0  4106.00  6132.5  13303.00  36958.0    3390.0   
D           8061.677206  632.0  4136.75  5800.0  13724.50  36958.0    1924.0   

                                                                      \
                 mean        std  min       25%       50%        75%   
holc_grade                                                             
A            1.628790   2.406080  0.0  0.194399  0.720414   2.219232   
B            0.976175   2.528996  0.0  0.031746  0.185874   0.742444   
C            1.630752   6.404026  0.0  0.017317  0.119966   0.557814   
D           12.363103  23.749602  0.0  0.069891  0.884296  10.225885   

                      pct_nonwhite                                       \
                  max        count       mean        std  min       25%   
holc_grade                                                                
A           14.559387        330.0   2.143227   3.484961  0.0  0.366972   
B           34.043134       1337.0   1.156934   2.836675  0.0  0.074479   
C           94.859155       3390.0   1.840216   6.423241  0.0  0.071777   
D           99.832003       1924.0  13.103528  23.874658  0.0  0.218606   

                                           pct_foreign_white             \
                 50%        75%        max             count       mean   
holc_grade                                                                
A           1.137912   2.632004  32.479627             330.0   9.533420   
B           0.343980   1.011271  47.159317            1337.0  13.256745   
C           0.295558   0.978085  94.859155            3390.0  16.176714   
D           1.705179  12.193057  99.832003            1924.0  19.151403   

                                                                             \
                  std       min        25%        50%        75%        max   
holc_grade                                                                    
A            6.646491  1.005414   4.317577   8.798307  12.291715  35.507212   
B            7.897680  0.497950   7.639272  11.749100  17.684867  40.001779   
C            8.956986  0.107527   9.709296  15.316742  21.484108  47.282815   
D           10.767988  0.206132  10.790095  19.111415  27.365985  51.851852   

           median_h_value                                             \
                    count      mean       std    min      25%    50%   
holc_grade                                                             
A                   330.0  8.823648  4.047674  1.080  5.95000  7.471   
B                  1337.0  6.072810  2.54